In [17]:
from dotenv import load_dotenv
load_dotenv()
import os
import openai
from pinecone import Pinecone, ServerlessSpec
import json

In [16]:
pc = Pinecone(api_key="b08cba53-157d-4428-b6e7-acec52f78f5e")
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [30]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Physics',
  'stars': 5,
  'review': "Dr. Johnson's lectures are engaging and informative. She explains complex concepts clearly and is always willing to help students outside of class."},
 {'professor': 'Professor Mark Thompson',
  'subject': 'History',
  'stars': 3,
  'review': 'Professor Thompson knows his material well, but his lectures can be a bit dry. Participation in class discussions helps a lot.'},
 {'professor': 'Dr. Sarah Lee',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Dr. Lee is a great teacher who makes math interesting. Her exams are challenging but fair, and she provides plenty of resources for studying.'},
 {'professor': 'Professor James Smith',
  'subject': 'Computer Science',
  'stars': 2,
  'review': 'Professor Smith is knowledgeable, but his teaching style is hard to follow. The lectures often feel disorganized.'},
 {'professor': 'Dr. Lisa Brown',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Dr. 

In [31]:
import openai
processed_data = []
client = openai.OpenAI()
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )


In [32]:
item = processed_data[0]
print(item)

{'values': [-0.004751737, -0.015690468, 0.031079438, 0.043817736, 0.004290068, 0.00050131924, 0.0040545226, 0.055978164, -0.011921741, 0.029496573, 0.03944601, -0.008046233, -0.027762959, 0.012549862, 0.004139319, 0.03736065, -0.03321505, -0.020125004, 0.03617978, 0.07497255, 0.0331648, -0.010018533, 0.026381092, -0.026481591, -0.019371258, -0.054018427, 0.007920609, 0.025602221, 0.033868298, -0.014836223, 0.08170601, -0.0006230177, -0.03507429, -0.02896895, -0.01233002, 0.03793852, 0.009968284, 0.00043183335, 0.01721052, 0.013592543, 0.003922617, -0.0013724449, -0.043189615, -0.0051537347, 0.036079284, 0.0069784266, 0.014233227, -0.012160427, 0.04926983, 0.04627997, -0.035149664, 0.006089635, 0.016381402, -0.029873446, -0.02585347, -0.019572258, 0.018542139, 0.036606904, -0.012028522, -0.007675641, 0.037611898, 0.0031563092, -0.027637335, -0.0015891467, -0.013856354, -0.013190546, -0.0034954946, -0.0018560982, -0.035677284, -0.042410746, 0.042008746, 0.0057441685, -0.05632991, 0.00726

In [33]:
index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")

Upserted count: 20


In [34]:
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}
